In [1]:
%%capture
!pip install selenium
!pip install webdriver-manager
!pip install pandas
!pip install bs4

In [2]:
from selenium import  webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time, re
import bs4
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta

service = Service(executable_path=r"C:\chromedriver\chromedriver.exe")

In [3]:
url = "https://www.lazada.co.th/products/3-i5432338284-s23195260672.html?pvid=95e452bf-d588-4c34-bec7-f84bf9de8bdc&search=jfy&scm=1007.45039.432958.0&priceCompare=skuId%3A23195260672%3Bsource%3Atpp-recommend-plugin-32104%3Bsn%3A95e452bf-d588-4c34-bec7-f84bf9de8bdc%3BoriginPrice%3A5900%3BdisplayPrice%3A5900%3BsinglePromotionId%3A-1%3BsingleToolCode%3AmockedSalePrice%3BvoucherPricePlugin%3A0%3Btimestamp%3A1746879709497&spm=a2o4m.homepage.just4u.d_5432338284" #รับ url ในอนาคต

def open_driver():
    options = Options()
    #options.add_argument(r"--user-data-dir=C:\Users\Thanb\AppData\Local\Google\Chrome\User Data")
    #options.add_argument("--profile-directory=Default")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    #options.add_argument("--headless")  # ไม่แสดงหน้าต่างเบราว์เซอร์
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(service=Service(), options=options)
    return driver

start_period = "2025-5"
end_period = "2025-05"

# แปลง start_period เป็น datetime แล้วลบ 1 เดือน
start_dt = datetime.strptime(start_period, "%Y-%m")
stop_dt = start_dt - relativedelta(months=1)

# แปลงกลับเป็น string ในรูปแบบ YYYY-MM
stop_period = stop_dt.strftime("%Y-%m")

next_button = "/html/body/div[5]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]" #ปุ่มหน้าถัดไป
sort_button_xpath = "/html/body/div[5]/div/div[10]/div[1]/div[2]/div/div/div/div[2]/div/div[2]" #ปุ่มเรียงลำดับ
newest_first_option_xpath = "/html/body/div[9]/div/div/ul/li[2]" #ปุ่มเรียงจากล่าสุด

# ฟังก์ชันเลื่อนหน้า (fix lazy loading)
def scroll_to_bottom(driver):
    prev_height = 0
    while True:
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(1)
        new_height = driver.execute_script("return window.scrollY + window.innerHeight")
        page_height = driver.execute_script("return document.body.scrollHeight")
        if new_height >= page_height:
            break

#click element
def click_element(driver, xpath):
    element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(1)

# เริ่มต้นเบราว์เซอร์
driver = open_driver()
driver.get(url)
time.sleep(3)
scroll_to_bottom(driver)# Scroll เพื่อให้ element โหลด

while True:
    try:
        # รอให้ปุ่ม Sort ปรากฏ
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, sort_button_xpath)))
        print("✅ เจอปุ่ม Sort")
        break  # ออกจาก loop ถ้าเจอปุ่ม Sort
    except:
        print("❌ ไม่เจอปุ่ม Sort — รีสตาร์ท Chrome")
        driver.quit()  # ปิด driver
        driver = open_driver()  # เปิด driver ใหม่
        driver.get(url)
        time.sleep(3)
        scroll_to_bottom(driver)

click_element(driver, sort_button_xpath)
click_element(driver, newest_first_option_xpath)

thai_months = {"ม.ค.": "01", "ก.พ.": "02", "มี.ค.": "03", "เม.ย.": "04", "พ.ค.": "05", "มิ.ย.": "06", "ก.ค.": "07", "ส.ค.": "08", "ก.ย.": "09", "ต.ค.": "10", "พ.ย.": "11", "ธ.ค.": "12"}

def to_full_date(text):
    now = datetime.now()
    if "ชั่วโมง" in text:
        dt = now - timedelta(hours=int(re.search(r'\d+', text).group()))
    elif "วัน" in text:
        dt = now - timedelta(days=int(re.search(r'\d+', text).group()))
    elif "สัปดาห์" in text:
        dt = now - timedelta(weeks=int(re.search(r'\d+', text).group()))
    elif "นาที" in text:
        dt = now - timedelta(minutes=int(re.search(r'\d+', text).group()))
    elif "วินาที" in text:
        dt = now - timedelta(seconds=int(re.search(r'\d+', text).group()))
    else:
        parts = text.split()
        if len(parts) == 3:
            day = parts[0]
            month = thai_months.get(parts[1], None)
            year = parts[2]
            if month:
                date_str = f"{year}-{month}-{day.zfill(2)}"
                try:
                    dt = datetime.strptime(date_str, "%Y-%m-%d")
                    return dt.strftime("%Y-%m-%d")
                except:
                    return None
        return None
    return dt.strftime("%Y-%m-%d")


def is_within_period(month_year):
    return start_period <= month_year <= end_period

def click_next_button():
    try:
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)
        return True
    except:
        return False

def get_star_rating(review):
    stars = review.find_all("img", class_="star")  # หา <img class="star">
    return len(stars) 

collect = False
all_reviews = []

print("🔄 เริ่มดึงรีวิว...")

while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    date_spans = soup.find_all("span", class_="title right")

    if not date_spans:
        break

    stop = False

    for span in date_spans:
        raw_date = span.get_text(strip=True)
        full_date = to_full_date(raw_date)
        month_year = full_date[:7] if full_date else None

        if month_year is None:
            continue

        # ✅ ยังไม่เริ่มเก็บ → รอจนเจอ <= end_period ก่อน
        if not collect:
            if month_year <= end_period:
                collect = True
                print(f"✅ เริ่มเก็บข้อมูลที่: {month_year}")

        # ✅ เริ่มเก็บแล้ว
        if collect:
            if month_year <= stop_period:
                stop = True
                break
            else:
                # หารีวิวจาก 'item-content'
                content_divs = [
                    content
                    for item in soup.find_all('div', class_='item-content')
                    if item.get('class') == ['item-content']
                    for content in item.find_all('div', class_='content')
                ]
                reviews = [div.get_text(strip=True) for div in content_divs]

                # หาจำนวนดาวจาก 'container-star starCtn left'
                star_divs = soup.find_all('div', class_='container-star starCtn left')
                star_ratings = [
                    len([img for img in div.find_all('img', class_='star') if img.get('src') == "//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"])
                    for div in star_divs
                ]

                # เก็บข้อมูลรีวิวพร้อมจำนวนดาว
                for review, rating in zip(reviews, star_ratings):
                    all_reviews.append({"ratings": rating, "text": review, "date": full_date})
                    

    if stop:
        print("✅ หยุดดึง: เจอเดือนก่อนช่วงเริ่มต้นแล้ว")
        break

    if not click_next_button():
        print("🛑 ไม่มีปุ่ม Next แล้ว")
        break


df = pd.DataFrame(all_reviews)

✅ เจอปุ่ม Sort
🔄 เริ่มดึงรีวิว...
✅ เริ่มเก็บข้อมูลที่: 2025-04
✅ หยุดดึง: เจอเดือนก่อนช่วงเริ่มต้นแล้ว


In [4]:
url = "https://www.lazada.co.th/products/3-i5432338284-s23195260672.html?pvid=95e452bf-d588-4c34-bec7-f84bf9de8bdc&search=jfy&scm=1007.45039.432958.0&priceCompare=skuId%3A23195260672%3Bsource%3Atpp-recommend-plugin-32104%3Bsn%3A95e452bf-d588-4c34-bec7-f84bf9de8bdc%3BoriginPrice%3A5900%3BdisplayPrice%3A5900%3BsinglePromotionId%3A-1%3BsingleToolCode%3AmockedSalePrice%3BvoucherPricePlugin%3A0%3Btimestamp%3A1746879709497&spm=a2o4m.homepage.just4u.d_5432338284" #รับ url ในอนาคต

def open_driver():
    options = Options()
    #options.add_argument(r"--user-data-dir=C:\Users\Thanb\AppData\Local\Google\Chrome\User Data")
    #options.add_argument("--profile-directory=Default")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    #options.add_argument("--headless")  # ไม่แสดงหน้าต่างเบราว์เซอร์
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(service=Service(), options=options)
    return driver

start_period = "2025-5"
end_period = "2025-05"

# แปลง start_period เป็น datetime แล้วลบ 1 เดือน
start_dt = datetime.strptime(start_period, "%Y-%m")
stop_dt = start_dt - relativedelta(months=1)

# แปลงกลับเป็น string ในรูปแบบ YYYY-MM
stop_period = stop_dt.strftime("%Y-%m")

next_button = "/html/body/div[5]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]" #ปุ่มหน้าถัดไป
sort_button_xpath = "/html/body/div[5]/div/div[10]/div[1]/div[2]/div/div/div/div[2]/div/div[2]" #ปุ่มเรียงลำดับ
newest_first_option_xpath = "/html/body/div[9]/div/div/ul/li[2]" #ปุ่มเรียงจากล่าสุด

# ฟังก์ชันเลื่อนหน้า (fix lazy loading)
def scroll_to_bottom(driver):
    prev_height = 0
    while True:
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(1)
        new_height = driver.execute_script("return window.scrollY + window.innerHeight")
        page_height = driver.execute_script("return document.body.scrollHeight")
        if new_height >= page_height:
            break

#click element
def click_element(driver, xpath):
    element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(1)

# เริ่มต้นเบราว์เซอร์
driver = open_driver()
driver.get(url)
time.sleep(3)
scroll_to_bottom(driver)# Scroll เพื่อให้ element โหลด

while True:
    try:
        # รอให้ปุ่ม Sort ปรากฏ
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, sort_button_xpath)))
        print("✅ เจอปุ่ม Sort")
        break  # ออกจาก loop ถ้าเจอปุ่ม Sort
    except:
        print("❌ ไม่เจอปุ่ม Sort — รีสตาร์ท Chrome")
        driver.quit()  # ปิด driver
        driver = open_driver()  # เปิด driver ใหม่
        driver.get(url)
        time.sleep(3)
        scroll_to_bottom(driver)

click_element(driver, sort_button_xpath)
click_element(driver, newest_first_option_xpath)

thai_months = {"ม.ค.": "01", "ก.พ.": "02", "มี.ค.": "03", "เม.ย.": "04", "พ.ค.": "05", "มิ.ย.": "06", "ก.ค.": "07", "ส.ค.": "08", "ก.ย.": "09", "ต.ค.": "10", "พ.ย.": "11", "ธ.ค.": "12"}

def to_full_date(text):
    now = datetime.now()
    if "ชั่วโมง" in text:
        dt = now - timedelta(hours=int(re.search(r'\d+', text).group()))
    elif "วัน" in text:
        dt = now - timedelta(days=int(re.search(r'\d+', text).group()))
    elif "สัปดาห์" in text:
        dt = now - timedelta(weeks=int(re.search(r'\d+', text).group()))
    elif "นาที" in text:
        dt = now - timedelta(minutes=int(re.search(r'\d+', text).group()))
    elif "วินาที" in text:
        dt = now - timedelta(seconds=int(re.search(r'\d+', text).group()))
    else:
        parts = text.split()
        if len(parts) == 3:
            day = parts[0]
            month = thai_months.get(parts[1], None)
            year = parts[2]
            if month:
                date_str = f"{year}-{month}-{day.zfill(2)}"
                try:
                    dt = datetime.strptime(date_str, "%Y-%m-%d")
                    return dt.strftime("%Y-%m-%d")
                except:
                    return None
        return None
    return dt.strftime("%Y-%m-%d")


def is_within_period(month_year):
    return start_period <= month_year <= end_period

def click_next_button():
    try:
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)
        return True
    except:
        return False

def get_star_rating(review):
    stars = review.find_all("img", class_="star")  # หา <img class="star">
    return len(stars) 

collect = False
all_reviews = []

print("🔄 เริ่มดึงรีวิว...")

while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # ดึงรีวิว, ดาว, วันที่ 1 ครั้งต่อหน้า
    content_divs = [
        content
        for item in soup.find_all('div', class_='item-content')
        if item.get('class') == ['item-content']
        for content in item.find_all('div', class_='content')
    ]
    reviews = [div.get_text(strip=True) for div in content_divs]

    star_divs = soup.find_all('div', class_='container-star starCtn left')
    star_ratings = [
        len([img for img in div.find_all('img', class_='star') if img.get('src') == "//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"])
        for div in star_divs
    ]

    date_spans = soup.find_all("span", class_="title right")
    raw_dates = [span.get_text(strip=True) for span in date_spans]
    full_dates = [to_full_date(d) for d in raw_dates]

    # ใช้จำนวนที่ตรงกันทั้ง 3
    min_len = min(len(reviews), len(star_ratings), len(full_dates))
    reviews = reviews[:min_len]
    star_ratings = star_ratings[:min_len]
    full_dates = full_dates[:min_len]

    stop = False

    for review, rating, full_date in zip(reviews, star_ratings, full_dates):
        if not full_date:
            continue
        month_year = full_date[:7]

        if not collect:
            if month_year <= end_period:
                collect = True
                print(f"✅ เริ่มเก็บข้อมูลที่: {month_year}")

        if collect:
            if month_year <= stop_period:
                stop = True
                break
            all_reviews.append({"ratings": rating, "text": review, "date": full_date})

    if stop:
        print("✅ หยุดดึง: เจอเดือนก่อนช่วงเริ่มต้นแล้ว")
        break

    if not click_next_button():
        print("🛑 ไม่มีปุ่ม Next แล้ว")
        break

df = pd.DataFrame(all_reviews)

✅ เจอปุ่ม Sort
🔄 เริ่มดึงรีวิว...
✅ เริ่มเก็บข้อมูลที่: 2025-04
✅ หยุดดึง: เจอเดือนก่อนช่วงเริ่มต้นแล้ว


In [5]:
df

""


In [6]:
df.to_csv("df_lazada_sentiment.csv", index=False, encoding='utf-8-sig')

In [7]:
%%capture
!pip install transformers datasets scikit-learn torch
!pip install seaborn
!pip install emoji
!pip install imbalanced-learn
!pip install xgboost

In [8]:
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import emoji
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_words
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import joblib

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
VOCAB = set(thai_words())

def clean_tokenize(text):
    text = emoji.replace_emoji(text, '').replace('\n', ' ').replace('\r', ' ') #ลบ emoji และการขึ้นบรรทัดใหม่
    text = re.sub(r'(.)\1{3,}', r'\1\1', text)  # ตัดอักษรซ้ำเกิน 2 ตัว
    text = re.sub(r'[^\u0E00-\u0E7Fa-zA-Z0-9\s:]', '', text)  # ลบอักขระพิเศษ
    seen = set()
    text = " ".join([w for w in text.split() if not (w in seen or seen.add(w))])  # ลบกลุ่มคำซ้ำในคอมเมนต์
    tokens = word_tokenize(text, engine="newmm", keep_whitespace=True) #tokenize
    return [w for w in tokens if w in VOCAB or w.isnumeric()] #กรองคำ

def safe_clean(text):
    try:
        # ตรวจสอบว่าเป็นสตริงและทำความสะอาด
        if isinstance(text, str):
            return clean_tokenize(text)
        else:
            return None  # กรณีที่ไม่ใช่สตริง
    except Exception as e:
        print(f"Error processing row: {e}")  # สามารถแสดงข้อผิดพลาดได้
        return None  # ข้ามไปทำงานต่อ

In [10]:
df["text_token"] = df["text"].apply(safe_clean)
df = df[~df['text_token'].isin(['[]', '', None])]
df

KeyError: 'text'

In [ ]:
# โหลดทั้งหมด
cvec = joblib.load('cvec.pkl')
le_sentiment = joblib.load('le_sentiment.pkl')
xgb_sentiment = joblib.load('xgb_sentiment.pkl')

le_category = joblib.load('le_category.pkl')
xgb_category = joblib.load('xgb_category.pkl')

In [ ]:
# vectorize
X_vec = cvec.transform(df['text_token'])

# ทำนาย sentimentType
y_sentiment_pred = xgb_sentiment.predict(X_vec)
df['sentimentType'] = le_sentiment.inverse_transform(y_sentiment_pred)

# ทำนาย commentCategoryName
y_category_pred = xgb_category.predict(X_vec)
df['commentCategoryName'] = le_category.inverse_transform(y_category_pred)

In [ ]:
#กำหนดชื่อคำใหม่ตามรูปแบบที่กำหนด
df['sentimentType'] = df['sentimentType'].replace({'pos': 'Positive', 'neg': 'Negative', 'neu': 'Neutral'})
df["commentCategoryName"] = df["commentCategoryName"].replace({'product': 'Product', 'delivery': 'Delivery', 'service': 'Service', 'none': 'Other'})
df = df.drop(columns=['text_token'])
df

,ratings,text,date,sentimentType,commentCategoryName
0,5,ดีๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ,2025-05-18,Positive,Product
1,5,ดี,2025-05-18,Positive,Product
2,5,คุณภาพ ตามราคา,2025-05-18,Neutral,Product
3,5,ดีตรงปก,2025-05-17,Positive,Product
4,5,ได้รับสินค้าแล้ว ทางร้านส่งสินค้าครบ ตรงตามที่...,2025-05-17,Neutral,Service
...,...,...,...,...,...
285,4,สมราคา\nส่งช้า,2025-05-04,Negative,Delivery
286,5,วางของได้เยอะ เหมาะสมกับราคา,2025-05-04,Positive,Product
287,5,🎨การออกแบบ:ดีมีสไตล์ คุ้มค่ากับเงินที่จ่าย ใช้...,2025-05-04,Positive,Product
288,5,นึกว่าอันใหญ่ที่ไหนได้เป็นอันเล็ก ไม่โอเคเลยค่...,2025-05-04,Negative,Product


In [ ]:
#ดูว่า sentimentType มีอารมณ์อะไรเท่าไหร่
df['sentimentType'].value_counts()

sentimentType
Positive    115
Neutral      88
Negative     87
Name: count, dtype: int64

In [ ]:
df['ratings'].value_counts().sort_index()

ratings
1      8
2      5
3     10
4     13
5    254
Name: count, dtype: int64

In [ ]:
#ดูว่า commentCategoryName มีเรื่องอะไรเท่าไหร่
df['commentCategoryName'].value_counts()

commentCategoryName
Product     206
Other        59
Delivery     15
Service      10
Name: count, dtype: int64

In [ ]:
#ส่วนนี้ กำหนดให้บันทึกผลลัพธืที่ได้มาเป็นไฟล์ csv 
df.to_csv('output.csv', index=False, encoding='utf-8-sig')